# Multiprocessing in Python

What happens if we need to do a lot of computation, and vectorization with NumPy is not enough? 

Basically: How can we assign different tasks to different cores or processors?

## Multiprocessing systems/interfaces

There are a few types of multiprocessing interfaces we can use, in order of easy to difficult:

| Interface | When to use |
|-----------|-------------|
| concurrent.futures.ProcessPoolExecutor | The modern way to launch parallel tasks. Usable for everything that takes a long time and needs to run on only one computer. |
| mpi4py.futures.MPIPoolExecutor | The modern way to run scalable parallel tasks on computer clusters. Use for long tasks that need to run on more than one computer. |
| multiprocess | An improved version of the built-in `multiprocessing` library. A bit more manual than the pools. |
| multiprocessing | The classic built-in library.  You probably will be using a part of this (like the Manager), but other components are trickier than the concurrent.futures implementation. |
| mpi4py classic | A bit beyond the scope of this class. |
